## 1. 데이터 전처리

runpod에서 a100 GPU 1개 sxm 대여해주시고 container disk는 50기가 바이트

In [2]:
%pip install --upgrade torch torchvision torchaudio
%pip install "transformers>=4.50.0" --force-reinstall --no-deps
%pip install tokenizers safetensors huggingface-hub --upgrade
%pip install datasets accelerate trl peft --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 128.0 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.22.0.dev20250319+cu128
    Uninstalling torchvision-0.22.0.dev20250319+cu128:
      Successfully uninstalled torchvision-0.22.0.dev20250319+cu128
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.6.0.dev20250319+cu128
    Uninstalling torchaudio-2.6.0.dev20250319+cu128:
      Successfully uninstalled torchaudio-2.6.0.dev20250319+cu128

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
Using cached transformers-4.55.2-py3-none-any.whl (11.3 MB)
  Attempting uninstall: transformers
    Found existing installation: transfor

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

In [2]:
# 1. 허깅페이스 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/esg-survey-datasets", split="train")

# 2. OpenAI format으로 데이터 변환을 위한 함수 
def format_data(sample):
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": sample['system_prompt'],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": sample["assistant"]
            },
        ],
    }

In [3]:
# 1. 허깅페이스 허브에서 데이터셋 로드
from datasets import load_dataset
import random

dataset = load_dataset("iamjoon/esg-survey-datasets", split="train")

# 2. OpenAI format으로 데이터 변환을 위한 함수 
def format_data(sample):
    # OpenAI format으로 변환
    return {
        "messages": [
            {
                "role": "system",
                "content": sample['system_prompt'],
            },
            {
                "role": "user",
                "content": sample["user_prompt"],
            },
            {
                "role": "assistant",
                "content": sample["assistant"]
            },
        ],
    }

# 3. 전체 데이터에 OpenAI 포맷 전처리 적용
formatted_dataset = dataset.map(format_data)

# 4. 데이터를 7:2 비율로 학습/테스트 분할 (HuggingFace 내장 메서드 사용)
split_dataset = formatted_dataset.train_test_split(test_size=0.3, seed=42)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

# 5. 결과 확인
print(f"전체 데이터 수: {len(formatted_dataset)}")
print(f"학습 데이터 수: {len(train_dataset)} ({len(train_dataset)/len(formatted_dataset)*100:.1f}%)")
print(f"테스트 데이터 수: {len(test_dataset)} ({len(test_dataset)/len(formatted_dataset)*100:.1f}%)")

Map:   0%|          | 0/4599 [00:00<?, ? examples/s]

전체 데이터 수: 4599
학습 데이터 수: 3219 (70.0%)
테스트 데이터 수: 1380 (30.0%)


In [4]:
# 6. 샘플 데이터 확인
print("\n=== 학습 데이터 샘플 ===")
for message in train_dataset[0]['messages']:
    print(f"[{message['role'].upper()}]")

    # 상위 500개 문자열만 출력
    print(message['content'][:500])
    print("-" * 50)


=== 학습 데이터 샘플 ===
[SYSTEM]
당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
ABOUT THIS REPORT
본 보고서는 고객, 직원, 주주, 투자자 등 이해관계자들에게 동국제강의 
환경경영 정보를 제공하기 위하여 발행하였습니다. 동국제강은 철강
기업으로서 ‘자원순환 사회’ 및 ‘저탄소 사회’ 실현에 공헌하는 것을 환경
과제로 삼고 있으며, 이러한 활동과 성과에 대한 정보를 중심으로 구성
하고 있습니다.
보고범위
동국제강 본사 및 국내 사업장(인천공장, 포항공장, 부산공장, 당진공장, 
신평공장)의 활동과 성과를 담고 있습니다.
보고기간
2020년 1월 1일 ~ 2020년 12월 31일 동안의 활동 내용을 주로 담고 
있으며, 환경개선 효과를 비교하기 위하여 과거 수 개년 동안의 실적과 
향후 목표를 함께 수록하였습니다.
본 보고서는 지속가능경영보고서의 국
--------------------------------------------------
[USER]
질문: C3-2-2. 지배구조 요소와 관련하여 아래 항목 중 현재 재직하고 있는 회사에 해당되는 응답을 선택해주세요._1)(상장기업만 응답) 공정공시제도를 준수하고 있는가?

보기:
['1) 전혀그렇지않다', '2) 약간그렇지않다', '3) 보통이다', '4) 약간그렇다', '5) 매우그렇다']
--------------------------------------------------
[ASSISTANT]
답변: 알 수 없음
--------------------------------------------------


In [5]:
print("\n=== 테스트 데이터 샘플 ===")
for message in test_dataset[0]['messages']:
    print(f"[{message['role'].upper()}]")

    # 상위 500개 문자열만 출력
    print(message['content'][:500])
    print("-" * 50)


=== 테스트 데이터 샘플 ===
[SYSTEM]
당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
ESG JOURNEY ESG IMPACT 2023 ESG PERFORMANCE APPENDIXCOMPANY OVERVIEW
대웅제약 2024 지속가능경영보고서
비상대응
비상대응체계 구축  
대웅제약은 화재, 누출 누유 및 안전/환경 재해, 태풍, 폭우, 지진 등 자연재해 그리고 정전 등을 사전에 	
예방하고 발생 시 신속하게 대응하기 위한 비상대응 체제를 구축 및 운영하여 체계적인 환경 및 안전보건 
경영을 실현해 나가고 있습니다. 이를 통해 사업장에 중대산업재해가 발생하거나 발생할 급박한 위험이 있을 
경우를 대비하여 사후조치에 관한 매뉴얼을 마련하고, 위기사항 등급에 따라 결정권자를 정하여 위기상황 
유형에 따라 정확하게 대처하며, 해당 매뉴얼에 대한 교육 및 훈련을 실시
--------------------------------------------------
[USER]
질문: C3-1-1. 지배구조 요소와 관련하여 아래 항목 중 현재 재직하고 있는 회사에 해당되는 응답을 선택해주세요._6)자금 집행과 관리 주체를 분리하여 체계적으로 운영하고 있는가?

보기:
['1) 전혀그렇지않다', '2) 약간그렇지않다', '3) 보통이다', '4) 약간그렇다', '5) 매우그렇다']
--------------------------------------------------
[ASSISTANT]
답변: 알 수 없음
--------------------------------------------------


In [6]:
# 리스트 형태에서 다시 Dataset 객체로 변경
print(type(train_dataset))
print(type(test_dataset))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [7]:
# 데이터셋 저장
test_dataset.save_to_disk("test_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/1380 [00:00<?, ? examples/s]

## 2. 모델 로드 및 템플릿 적용


In [8]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-8B" 

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [9]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
ABOUT THIS REPORT
본 보고서는 고객, 직원, 주주, 투자자 등 이해관계자들에게 동국제강의 
환경경영 정보를 제공하기 위하여 발행하였습니다. 동국제강은 철강
기업으로서 ‘자원순환 사회’ 및 ‘저탄소 사회’ 실현에 공헌하는 것을 환경
과제로 삼고 있으며, 이러한 활동과 성과에 대한 정보를 중심으로 구성
하고 있습니다.
보고범위
동국제강 본사 및 국내 사업장(인천공장, 포항공장, 부산공장, 당진공장, 
신평공장)의 활동과 성과를 담고 있습니다.
보고기간
2020년 1월 1일 ~ 2020년 12월 31일 동안의 활동 내용을 주로 담고 
있으며, 환경개선 효과를 비교하기 위하여 과거 수 개년 동안의 실적과 
향후 목표를 함께 수록하였습니다.
본 보고서는 지속가능경영보고서의 국제표준 가이드라인인 GRI(Global 
Reporting Initiative) Standards, TCFD 등의 환경부문에 대한 기준을 
반영하여 작성하였습니다. 보고서 내 재무 성과는 한국채택국제회계기
준(K-IFRS)으로 작성하였습니다.
서울특별시 중구 을지로5길 19(수하동, 페럼타워)
문의 02-317-1114   
홈페이지 www.dongkuk.com
철을 되살리다
생명을 되살리다
---
</document 0>

<document 1>
---
28
ENVIRONMENT REPORT 2021
BETTER LIFE WITH STEEL 
01 
SCR 투자
 -
동국제강은 정부의 친환경정책에 적극적으로 동참하여, 철강 생산과정에서 발생하는 질소산화물의 배출을 저감하고
자 2020년 부산에 소재한 신평공장의 가열로에 SCR설비를 도입하여 질소산화물 배출을 약 80% 저감시키는 등 SCR 
설비의 효과를 검증한 바 있습니다. 이에 추가적으로 포항

## 3. LoRA와 SFTConfig 설정


In [10]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

- lora_alpha: LoRA(Low-Rank Adaptation)에서 사용하는 스케일링 계수를 설정합니다. LoRA의 가중치 업데이트가 모델에 미치는 영향을 조정하는 역할을 하며, 일반적으로 학습 안정성과 관련이 있습니다.
- lora_dropout: LoRA 적용 시 드롭아웃 확률을 설정합니다. 드롭아웃은 과적합(overfitting)을 방지하기 위해 일부 뉴런을 랜덤하게 비활성화하는 정규화 기법입니다. 0.1로 설정하면 학습 중 10%의 뉴런이 비활성화.
- r: LoRA의 랭크(rank)를 설정합니다. 이는 LoRA가 학습할 저차원 공간의 크기를 결정합니다. 작은 값일수록 계산 및 메모리 효율이 높아지지만 모델의 학습 능력이 제한될 수 있습니다.
- bias: LoRA 적용 시 편향(bias) 처리 방식을 지정합니다. "none"으로 설정하면 편향이 LoRA에 의해 조정되지 않습니다. "all" 또는 "lora_only"와 같은 값으로 변경하여 편향을 조정할 수도 있습니다.
- target_modules: LoRA를 적용할 특정 모듈(레이어)의 이름을 리스트로 지정합니다. 예제에서는 "q_proj"와 "v_proj"를 지정하여, 주로 Self-Attention 메커니즘의 쿼리와 값 프로젝션 부분에 LoRA를 적용합니다.
- task_type: LoRA가 적용되는 작업 유형을 지정합니다. "CAUSAL_LM"은 Causal Language Modeling, 즉 시퀀스 생성 작업에 해당합니다. 다른 예로는 "SEQ2SEQ_LM"(시퀀스-투-시퀀스 언어 모델링) 등이 있습니다.

In [27]:
# 최대 길이
max_seq_length=16384

In [28]:
args = SFTConfig(
    output_dir="qwen3-8b-esg-survey-model",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=2,                      # 학습할 총 에포크 수 
    per_device_train_batch_size=1,           # GPU당 배치 크기
    gradient_accumulation_steps=8,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None,
    max_length=max_seq_length,              # 최대 시퀀스 길이 추가
)

- `output_dir`: 학습 결과가 저장될 디렉토리 또는 모델 저장소의 이름을 지정합니다. 이 디렉토리에 학습된 모델 가중치, 설정 파일, 로그 파일 등이 저장됩니다.

- `num_train_epochs`: 모델을 학습시키는 총 에포크(epoch) 수를 지정합니다. 에포크는 학습 데이터 전체를 한 번 순회한 주기를 의미합니다. 예를 들어, `3`으로 설정하면 데이터셋을 3번 학습합니.

- `per_device_train_batch_size`: GPU 한 대당 사용되는 배치(batch)의 크기를 설정합니다. 배치 크기는 모델이 한 번에 처리하는 데이터 샘플의 수를 의미합니다. 작은 크기는 메모리 사용량이 적지만 학습 시간이 증가할 수 있니다.

- `gradient_accumulation_steps`: 그래디언트를 누적할 스텝(step) 수를 지정합니다. 이 값이 `2`로 설정된 경우, 두 스텝마다 그래디언트를 업데이트합니다. 배치 크기를 가상으로 늘리는 효과가 있으며, GPU 메모리 부족 문제를 해결할 때 용합니다.

- `gradient_checkpointing`: 그래디언트 체크포인팅을 활성화하여 메모리를 절약합니다. 이 옵션은 계산 그래프를 일부 저장하지 않고 다시 계산하여 메모리를 절약하지만, 속도가 약간 느려질수 있습니다.

- `optim`: 학습 시 사용할 최적화 알고리즘을 설정합니다. `adamw_torch_fused`는 PyTorch의 효율적인 AdamW 최적기를 사용합니다.

- `logging_steps`: 로그를 기록하는 주기를 스텝 단위로 지정합니다. 예를 들어, `10`으로 설정하면 매 10 스텝마 로그를 기록합니다.

- `save_strategy`: 모델을 저장하는 전략을 설정합니다. `"steps"`로 설정된 경우, 지정된 스마다 모델이 저장됩니다.

- `save_steps`: 모델을 저장하는 주기를 스텝 단위로 설정합니다. 예를 들어, `50`으로 설정하면 매 50스텝마다 모델을 저장합니다.

- `bf16`: `bfloat16` 정밀도를 사용하도록 설정합니다. `bfloat16`은 FP32와 유사한 범위를 제공하면서 모리와 계산 효율성을 높입니다.

- `learning_rate`: 학습률을 지정합니다. 학습률은 모델의 가중치가 한 번의 업데이트에서 얼마나 크게 변할지를 결정합니다. 일반적으로 작은 값을 용하여 안정적인 학습을 유도합니다.

- `max_grad_norm`: 그래디언트 클리핑의 임계값을 설정합니다. 이 값보다 큰 그래디언트가 발생하면, 임계값으로 정하여 폭발적 그래디언트를 방지합니다.

- `warmup_ratio`: 학습 초기 단계에서 학습률을 선형으로 증가시키는 워밍업 비율을 지정합니다 학습의 안정성을 높이기 위해 사용됩니다.

- `lr_scheduler_type`: 학습률 스케줄러의 유형을 설정합니다. `"costant"`는 학습률을 일정하게 유지합니다.

- `push_to_hub`: 학습된 모델을 허브에 업로드할지 여부를 설정합니. `False`로 설정하면 업로드하지 않습니다.

- `remove_unused_columns`: 사용되지 않는 열을 제거할지 여부를 설정합니다.`True`로 설정하면 메모리를 절약할 수 있습니다.

- `dataset_kwargs`: 데이터셋 로딩 시 추가적인 설정을 전달합니다. 예제에서는 `skip_prepare_dataset True`로 설정하여 데이터셋 준비 단계를 건너뜁니다.

- `report_to`: 학습 로그를 보고할 대상을 지정합니다. `None`으로 설정되면 로그가 기록되지 않습니다.

## 4. 학습 중 전처리 함수: collate_fn


In [29]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

collate_fn(batch) 함수는 자연어 처리 모델 학습을 위해 데이터를 전처리하는 역할을 수행합니다. 이 함수는 배치 내의 데이터를 처리하여 모델이 사용할 수 있는 입력 형식으로 변환합니다.

먼저, 각 샘플의 메시지에서 개행 문자를 제거하고 필요한 정보만 남깁니다. 정리된 메시지로 텍스트를 구성하고 이를 토큰화하여 input_ids와 attention_mask를 생성합니다. 이후 assistant 답변 부분을 찾아 해당 범위에 레이블을 설정합니다. 이 범위를 제외한 나머지 위치는 -100으로 설정하여 손실 계산에서 제외되도록 합니다.

최종적으로, 배치 내 모든 샘플의 길이를 동일하게 맞추기 위해 패딩 작업을 수행합니다. 이 과정에서 입력 데이터에는 패딩 토큰 ID를 추가하고, 어텐션 마스크에는 0을 추가하며, 레이블에는 -100을 추가합니다. 모든 데이터는 PyTorch 텐서로 변환되어 반환됩니다.

In [30]:
# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 8907])
어텐션 마스크 형태: torch.Size([1, 8907])
레이블 형태: torch.Size([1, 8907])


In [31]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 55673, 31079, 85251, 138520, 19391, 130869, 85322, 77226, 98801, 11, 138520, 11, 63332, 131777, 81718, 144059, 42039, 143604, 129264, 130650, 624, 129845, 77226, 98801, 18411, 81718, 144059, 42039, 143604, 47836, 28733, 130768, 134832, 364, 144135, 28733, 46682, 48431, 6, 130939, 135968, 33509, 50302, 382, 129845, 77226, 98801, 510, 27, 6062, 220, 15, 397, 10952, 75530, 10039, 44076, 198, 125822, 63332, 34395, 26698, 16560, 138501, 11, 137351, 54321, 11, 55673, 54330, 11, 10764, 230, 105, 25715, 25715, 77002, 133329, 130891, 25715, 132812, 126322, 124785, 37087, 130262, 20401, 715, 132892, 142408, 60039, 18411, 130094, 66425, 133621, 95996, 124528, 143959, 13, 126322, 124785, 37087, 130262, 33704, 48364, 254, 130262, 198, 131754, 42039, 26698, 3369, 25715, 54321, 130237, 65238, 130592, 527, 128355, 3369, 126781, 135356, 43590, 130592, 527, 126423, 126407, 19391, 125466, 136770, 42905, 129337, 46832, 246, 65306, 198, 53680, 3708

In [32]:
# 디코딩된 input_ids 출력
decoded_text = tokenizer.decode(
    batch["input_ids"][0].tolist(),
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\ninput_ids 디코딩 결과:")
print(decoded_text)


input_ids 디코딩 결과:
<|im_start|>system
당신은 주어진 질문에 대해서 검색 결과, 질문, 보기를 바탕으로 답변해야 합니다.
검색 결과를 바탕으로 답변할 수 없는 경우에는 '알 수 없음'이라고 적으세요.

검색 결과:
<document 0>
---
ABOUT THIS REPORT
본 보고서는 고객, 직원, 주주, 투자자 등 이해관계자들에게 동국제강의 
환경경영 정보를 제공하기 위하여 발행하였습니다. 동국제강은 철강
기업으로서 ‘자원순환 사회’ 및 ‘저탄소 사회’ 실현에 공헌하는 것을 환경
과제로 삼고 있으며, 이러한 활동과 성과에 대한 정보를 중심으로 구성
하고 있습니다.
보고범위
동국제강 본사 및 국내 사업장(인천공장, 포항공장, 부산공장, 당진공장, 
신평공장)의 활동과 성과를 담고 있습니다.
보고기간
2020년 1월 1일 ~ 2020년 12월 31일 동안의 활동 내용을 주로 담고 
있으며, 환경개선 효과를 비교하기 위하여 과거 수 개년 동안의 실적과 
향후 목표를 함께 수록하였습니다.
본 보고서는 지속가능경영보고서의 국제표준 가이드라인인 GRI(Global 
Reporting Initiative) Standards, TCFD 등의 환경부문에 대한 기준을 
반영하여 작성하였습니다. 보고서 내 재무 성과는 한국채택국제회계기
준(K-IFRS)으로 작성하였습니다.
서울특별시 중구 을지로5길 19(수하동, 페럼타워)
문의 02-317-1114   
홈페이지 www.dongkuk.com
철을 되살리다
생명을 되살리다
---
</document 0>

<document 1>
---
28
ENVIRONMENT REPORT 2021
BETTER LIFE WITH STEEL 
01 
SCR 투자
 -
동국제강은 정부의 친환경정책에 적극적으로 동참하여, 철강 생산과정에서 발생하는 질소산화물의 배출을 저감하고
자 2020년 부산에 소재한 신평공장의 가열로에 SCR설비를 도입하여 질소산화물 배출을 약 80% 저감시키는 등 SCR 
설비의 효과를 검증한 

In [33]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [34]:
# -100이 아닌 부분만 골라 디코딩
label_ids = [token_id for token_id in batch["labels"][0].tolist() if token_id != -100]

decoded_labels = tokenizer.decode(
    label_ids,
    skip_special_tokens=False,
    clean_up_tokenization_spaces=False
)

print("\nlabels 디코딩 결과 (-100 제외):")
print(decoded_labels)


labels 디코딩 결과 (-100 제외):

<think>

</think>

답변: 알 수 없음<|im_end|>


### input_ids와 labels는 어떻게 생성되는가?


LLM 학습에서 `input_ids`와 `labels`는 모델의 학습 목표에 따라 생성됩니다. 이를 예시 문장과 정수 인코딩을 통해 상세히 설명하겠습니다.

예를 들어, 다음과 같은 대화 데이터를 모델이 학습해야 한다고 가정합니다.
사용자가 `안녕하세요, 오늘 날씨는 어떤가요?`라고 물었고,
모델은 `안녕하세요! 오늘 날씨는 맑고 화창합니다.`라고 응답해야 한다고 합시다.

LLaMA 3에서는 다음과 같은 템플릿 구조를 사용합니다:

`<|begin_of_text|><|start_header_id|>user<|end_header_id|>
안녕하세요, 오늘 날씨는 어떤가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
안녕하세요! 오늘 날씨는 맑고 화창합니다.<|eot_id|>`

이 전체 텍스트는 토크나이저에 의해 정수 시퀀스로 변환됩니다. 예시로 단순화된 정수 시퀀스는 다음과 같다고 가정합니다:

`input_ids = [1001, 2001, 3001, 4001, 5001, 6001, 7001, 1002, 1001, 8001, 9001, 1003, 2002]`

여기서 모델이 예측해야 할 영역은 assistant의 응답 부분인
`안녕하세요! 오늘 날씨는 맑고 화창합니다.`에 해당하는 토큰들입니다.
따라서 `labels`는 다음과 같이 설정됩니다:

`labels = [-100, -100, -100, -100, -100, -100, -100, -100, -100, 8001, 9001, 1003, 2002]`

이처럼 `labels`는 모델의 출력이 필요한 영역만을 포함하고, 나머지 부분은 `-100`으로 채워져
모델이 실제로 예측하고 오차를 계산해야 하는 대상(학습 대상)에서 제외됩니다.

이를 통해 모델은 불필요한 입력 부분을 학습하지 않고, assnt 응답 부분에만 집중할 수 있습니다.
"""


## 5. 학습

In [36]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [37]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,1.239100
20,0.283600
30,0.065500
40,0.050500
50,0.077800
60,0.041700
70,0.087500
80,0.066700
90,0.046200
100,0.064800
